# Uniswap Decentralized Exchange Pool Simulation

Owner: April Nellis

Companion code to [*DEX Specs: A Mean-Field Approach to DeFi Cryptocurrency Exchanges*](https://arxiv.org/abs/2404.09090)

In [ ]:
import numpy as np
import pandas as pd
import math
import itertools
pd.options.mode.chained_assignment = None  # default='warn'
import os.path
from datetime import datetime, timedelta
import time
import requests
import re
import statsmodels.api as sm
import scipy.stats as stats
from scipy.optimize import curve_fit
from scipy.stats import wasserstein_distance
from scipy import optimize
from IPython.display import Image
from coinmetrics.api_client import CoinMetricsClient
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import import_ipynb
from IPython.display import IFrame
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_error as mse
from sklearn.neighbors import KernelDensity
from decimal import Decimal

from sklearn.linear_model import LinearRegression, Ridge, Lasso

# Just disables the annoying warning, doesn't enable AVX/FMA
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

%matplotlib widget

#plt.rcParams["figure.autolayout"] = True
plt.rcParams["figure.figsize"] = [7,5]

## Initialization

This code is written for the specific example provided in DEX Specs. If the user wishes to examine a different pool or a different time interval, follow the following stpes:
1. Run `uniswapAnalysis.ipynb` for your desired pool and dates.
2. Adjust the `zeta` and `capital_list` parameters (below) accordingly.
3. Redefine the swapper's behavior in DEX.py if necessary.
4. Download the liquidity distribution at desired point(s) in time. (This can be achieved in a variety of ways, none of which are free.)
5. Convert the liquidity pool information into a `Pool` object (defined in DEX.py)

[**USDC/ETH Pool with 0.05% Fee**](https://info.uniswap.org/#/pools/0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640): This pool facilitates swaps between USDC and Ethereum. This pool has the largest daily trading volume and a much higher TVL than other USDC/ETH pools (as of 01/24/2023).

In [ ]:
# Contract addresses for each coin:
ETH = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'
USDC = '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'
POOL = 3

Load pool-specific information

In [ ]:
(TOK_A, TOK_A_ADR, TOK_B, TOK_B_ADR, FEE, PROTOCOL_STR, PROTOCOL) = ('ETH', ETH, 'USDC', USDC, '005', 'Uniswap V3: USDC 3', '0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640')

In [ ]:
NFT = '0xc36442b4a4522e871399cd717abdd847ab11fe88' # Uniswap V3: Positions NFT (tracks liquidity position)
ROUTER = '0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45|0xe592427a0aece92de3edee1f18e0157c05861564|0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b' # Uniswap V3: Router 2, facilitates exchanges

In [ ]:
start_date = datetime(2023, 11, 29, 0, 0, 0) # year, month, day, hour, minute, second
end_date = datetime(2023, 12, 1, 0, 0, 0)

If you change the dates, input your API key and run the following code:

In [ ]:
API_KEY = '' # Input personal Etherscan API Key (free)

In [ ]:
start_str = start_date.strftime('%Y-%m-%d')
start_unix = int(time.mktime(start_date.timetuple()))

my_url = f'https://api.etherscan.io/api?module=block&action=getblocknobytime&timestamp={start_unix}&closest=before&apikey={API_KEY}'

response = requests.get(my_url)
resp = response.json()
start_block = int(resp['result'])

end_str = end_date.strftime('%Y-%m-%d')
end_unix = int(time.mktime(end_date.timetuple()))

my_url = f'https://api.etherscan.io/api?module=block&action=getblocknobytime&timestamp={end_unix}&closest=before&apikey={API_KEY}'

response = requests.get(my_url)
resp = response.json()
end_block = int(resp['result'])
print(f'The period from {start_date} to {end_date} encompasses blocks {start_block} to {end_block} ({end_block-start_block} blocks).')

Load the CSV file containing desired Uniswap transaction data:

In [ ]:
master_string = 'data/pool3_master_18675043_18689338.csv'
#master_string = f'data/pool{POOL}_master_{start_block}_{end_block}.csv' # uncomment if changing dates or pool
master = pd.read_csv(master_string)
master.drop('Unnamed: 0', axis = 1, inplace = True)
master['timeStamp'] = pd.to_datetime(master['timeStamp'])
master['Exchange Rate'].replace([np.inf, -np.inf], np.nan, inplace=True) # just in case
master

## Comparing Uniswap Data with Simulation Results

In [ ]:
# parameters from previous Uniswap data analysis
capital_list = np.array([2124, 35786, 1706034])

In [ ]:
# parameters shared by all scenarios
myT = 24*60 # how many minutes to simulate over
myM = 300 # how many Monte Carlo simulations to run

In [ ]:
# load all the necessary functions so they don't clog the notebook
%run DEX.py
%run dexFunctions.py

In [ ]:
# load pre-saved liquidity distribution for November 29, 2023
date = '11292023'
curr_price = 2034.53333
intervals = np.load(f'data/{date}ticks.npy')
tokA = np.load(f'data/{date}tokA.npy')
tokB = np.load(f'data/{date}tokB.npy')
nu = Pool(pool_type = len(intervals), mkt = curr_price, pool = curr_price, fee = 0.0005)
nu.setTicks(intervals)
nu.manualLiqSetAll(tokA, tokB)
nu.setABTicks()

In [ ]:
ell_vector = np.zeros(nu.size)
for i in range(nu.size):
    ell_vector[i] = nu.K(i)
nu.plotEll(ell_vector, yLim = 5e7)

In [ ]:
nu.plotLiquidity(yLim = [500, 1e6])

### N-Player Bayesian Game Scenario

In [ ]:
# first, big position matrix
actions = np.zeros((int(nu.size*(nu.size+1)/2*3)+1, nu.size))
count = 0
for i in range(nu.idx):
    for j in range(nu.idx+1, nu.size+1):
        actions[count, i:j] = 1e2
        count +=1
print(count)
actions = np.transpose(actions[:count])
print(actions.shape)

In [ ]:
# next, let's find a good distribution for actions (let's use linreg to minimize number of actions):
reg_np = LinearRegression(positive=True, fit_intercept = False) # options are: Lasso, Ridge, LinearRegression
weight_np = reg_np.fit(actions, ell0).coef_
print(f'R-score is {reg_np.score(actions, ell0)}')

In [ ]:
actions2 = actions[:, weight_np > 0]
actions2.shape

In [ ]:
weights_np2 = weight_np[weight_np > 0]

In [ ]:
# Generate fee distributions for current liquidity distribution associated with nu
fees0,_ = V(-1, myT, nu, M=myM) # delta = -1
fees1,_ = V(0, myT, nu, M=myM) # delta = 0
fees2,_ = V(1, myT, nu, M=myM) # delta = 1

In [ ]:
# Approximate a CDF for fees that can be extrapolated to determine the fee distribution for any liquidity distribution
cdf0 = cdf_approx(fees0, nu) 
cdf1 = cdf_approx(fees1, nu)
cdf2 = cdf_approx(fees2, nu)
cdf_stack = np.stack([cdf0, cdf1, cdf2])
cdf_stack.shape

In [ ]:
# now, draw N samples from actions3 based on weights_np2 and do that many many many times
actionsM = 100
N = 10 # number of players
samples = np.random.choice(a = len(weights_np2), p = weights_np2/np.sum(weights_np2), size = (actionsM, N-1)) # sample the (N-1)-dimensional vector of other players' actions

In [ ]:
# Now, for each type, we must find the position that optimizes the expected payoff taken over all the samples
lambda_list = np.concatenate([np.linspace(0, 0.09, 10), np.arange(1, 3, 0.1)])

start = time.time()
count = 0
responses = np.zeros((3*3*len(lambda_list), 4)) #j1, j2, k, lambda, delta
for delta in [-1, 0, 1]:
    for k in capital_list*100:
        for lam in lambda_list:
            agent = agentLP(nu.size, belief = delta, risk = lam, capital = k)
            action_idx, v = br_N(agent, actions2, samples, cdf_stack, nu)
            responses[count] = [action_idx, k, lam, delta] # action, k, lam, delta, prob
            count += 1
end = time.time()

In [ ]:
print(f'The calibration took {(end - start)/60} minutes.')

In [ ]:
# calculate how many different types have each action as their best response
action_freq = np.zeros_like(weights_np2)
for c in range(count):
    idx = int(responses[c, 0])
    i = np.argmax(actions2[:,idx] > 0) # first nonzero index
    j = nu.size - np.argmax(np.flip(actions2[:,idx]) > 0) # one after last nonzero index
    
    k = responses[c,1]
    unit_cap = k/(j-i)
    action_freq[idx] += unit_cap
    
# calculate the probability to assign to each type theta
theta_w = np.zeros(count)
new_ell = np.zeros(nu.size)

for c in range(count):
    idx = int(responses[c, 0])
    
    i = np.argmax(actions2[:,idx] > 0) # first nonzero index
    j = nu.size - np.argmax(np.flip(actions2[:,idx]) > 0) # one after last nonzero index
    
    k = responses[c,1]
    unit_cap = k/(j-i)
    
    theta_w[c] = weights_np2[idx] * (unit_cap/action_freq[idx]) # assign probability to theta
    new_ell += theta_w[c] * actions2[:, idx] # add to liquidity distribution

In [ ]:
responses = np.concatenate((responses, np.expand_dims(theta_w, axis = 1)), axis = 1)

In [ ]:
selected_actions = actions2[:, action_freq>0]
selected_weights = weights_np2[action_freq>0]

scaled_weights = selected_weights*np.sum(weights_np2)/np.sum(selected_weights)

In [ ]:
ell_np = np.matmul(selected_actions, scaled_weights)

In [ ]:
print(f'MAPE is {mape(ell0, ell_np)} and Wasserstein is {wasserstein_distance(nu.ticks[1:], nu.ticks[1:], np.minimum(ell0, 8e5), np.minimum(ell_np, 8e5))}')

In [ ]:
nu.plotEll(ell_np, yLim = 1e6)

Estimate $\Theta$ for N-player Bayesian Game

In [ ]:
plus_dict = responses[responses[:,-1] > 0]

In [ ]:
plot_x = np.arange(0, np.amax(lambda_list), 0.1) # list of points that we will plug into density at end

plt.close()

fig, axs = plt.subplots(3,3)
i = 0
j = 0
joint_prob = np.zeros((3,3))

for delta in [-1, 0, 1]:
    j = 0
    for k in capital_list*100:
        selection = plus_dict[(plus_dict[:, 3] == delta)*(plus_dict[:, 1] == k)]
        joint_prob[j, i] = np.sum(selection[:, -1])/np.sum(plus_dict[:,-1])
        print(f'(k,d) = {(k, delta)} with frequency {len(selection)} and probability {joint_prob[j, i]}')
        fit_x = np.repeat(selection[:,2], (selection[:, -1]*1000).astype(int))
        if len(fit_x) > 1:
            kernel = stats.gaussian_kde(fit_x, bw_method = 0.1/fit_x.std(ddof=1))
            dens = kernel(plot_x)
            #kern = KernelDensity(kernel='gaussian', bandwidth=0.7).fit(fit_x[:, np.newaxis])
            #dens = np.exp(kern.score_samples(plot_x[:, np.newaxis]))
            axs[j,i].plot(plot_x, dens)
            axs[j,i].set_title(f'k = {int(k/100)*100}, delta = {delta}')
        else:
            print('too small')
        j+= 1
    i += 1
axs[-1,0].set_xlabel(r'$\lambda$')
axs[-1, 0].set_ylabel('Density')
fig.suptitle(r'Estimated density of $\lambda$ given ($k$, $\delta$)')
plt.show()

In [ ]:
# Probability distribution over capital endowments
np.sum(joint_prob, axis = 1)

Now let us try to play a game between agents:

In [ ]:
nu_np = nu.copy()

In [ ]:
nu_np.plotLiquidity()

In [ ]:
# agent1 is used in MFG
base_risk = 1e-6
#risk_list = np.array([base_risk, base_risk, base_risk, -1, base_risk/5, base_risk/5, base_risk, base_risk, base_risk])
risk_list = np.array([base_risk, base_risk, base_risk, base_risk/10, base_risk/5, base_risk/5, base_risk/2, base_risk/5, base_risk/2])
agent1 = agentLP(nu.size, belief = 1, risk = risk_list[8], capital = capital_list[0]*100)
agent2 = agentLP(nu.size, belief = 0, risk = risk_list[0], capital = capital_list[0]*100) 
agent3 = agentLP(nu.size, belief = 1, risk = risk_list[1], capital = capital_list[0]*100) 
agent4 = agentLP(nu.size, belief = 1, risk = risk_list[2], capital = capital_list[0]*100) 
agent5 = agentLP(nu.size, belief = 0, risk = risk_list[3], capital = capital_list[0]*100)
agent6 = agentLP(nu.size, belief = 1, risk = risk_list[4], capital = capital_list[0]*100) 
agent7 = agentLP(nu.size, belief = 1, risk = risk_list[5], capital = capital_list[0]*100)
agent8 = agentLP(nu.size, belief = 0, risk = risk_list[6], capital = capital_list[0]*100) 
agent9 = agentLP(nu.size, belief = 1, risk = risk_list[7], capital = capital_list[0]*100) 
agent10 = agentLP(nu.size, belief = 1, risk = risk_list[8], capital = capital_list[0]*100)

agents = [agent1, agent2, agent3, agent4, agent5, agent6, agent7, agent8, agent9, agent10]

In [ ]:
for ag in agents:
    ag.a = 0
    ag.b = nu.size
    ag.position = np.ones(nu.size)*ag.unit_liq(0, nu_np.size)
ell_init = findEll(nu_np.size, agents)
#nu.plotEll(ell_init)

In [ ]:
nu_np.resetLiquidity(ell_init, nu_np.pool_er)
print(nu_np.b_ticks[0], nu_np.b_ticks[-1])

In [ ]:
start = time.time()
ell_star = equilibrium(nu_np, agents, cdf_stack)
duration = (time.time() - start)/60

In [ ]:
print(f'Calculating the equilibrium distribution took {duration} minutes.')

In [ ]:
nu_np.resetLiquidity(ell_star, nu.pool_er)
nu_np.plotLiquidity(yLim = [500, 1e6])

In [ ]:
nu_np.plotEll(ell_star)

### MFG Scenario

Define a function to calibrate the distribution of agents based on a "real" distribution:

In [ ]:
ell0 = nu.tokB + nu.pool_er * nu.tokA
start = time.time()
fees, positions = calibrateAgents(nu, 0, myT, myM)
# fees, positions = calibrateAgents(nu, 0, myT, myM, loadFile = False) # if you want to generate new MC trials
print(f'Total time: {(time.time() - start)/60} minutes.')

In [ ]:
# make a positions matrix
scale_factor = 1e4

pos_mat = np.zeros((1, nu.size))
pos_mat[0, :] = capital_list[0]/nu.size
for i in range(1, positions.shape[0]):
    temp = np.zeros((1, nu.size))
    start = int(positions[i, 0])
    end = int(positions[i, 1])
    k = positions[i, 2]
    temp[0, start:end] = k/(end - start)
    pos_mat = np.concatenate((pos_mat, temp), axis = 0)
# new below
temp = np.ones((positions.shape[0], 1))*scale_factor
pos_mat = np.concatenate((pos_mat, temp), axis = 1)
ell_target = np.concatenate((ell0, [800*scale_factor]))
pos_mat = np.transpose(pos_mat) # pos_mat.shape = (nu.size, number of types)

reg_nnls = Ridge(positive=True, fit_intercept = False) # options are: Lasso, Ridge, LinearRegression
weights = reg_nnls.fit(pos_mat, ell_target).coef_
positions = np.concatenate((positions, np.expand_dims(weights, axis = 1)), axis = 1)

reg_nnls = LinearRegression(positive=True, fit_intercept = False) # options are: Lasso, Ridge, LinearRegression
weights = reg_nnls.fit(pos_mat, ell_target).coef_
print(f'R-score is {reg_nnls.score(pos_mat, ell_target)}')

ell_new = np.matmul(pos_mat, weights)
num_agents = ell_new[-1]
ell_solved = ell_new[:-1]
print(num_agents/scale_factor)

In [ ]:
print(f'Wasserstein distance is {wasserstein_distance(nu.ticks[1:], nu.ticks[1:], ell0, ell_solved)}')

In [ ]:
nu_temp = nu.copy()
nu_temp.resetLiquidity(ell_solved, nu.pool_er)
liq_mfg = np.zeros(nu.size)
for i in range(nu.size):
    liq_mfg[i] = nu_temp.K(i)
del nu_temp

In [ ]:
nu.plotEll(liq_mfg, yLim = 5e7)

Estimate $\Theta$ for MFG scenario

In [ ]:
plus_dict = positions[positions[:,-1] > 0]

In [ ]:
plot_x = np.arange(0, 6, 0.1) # list of points that we will plug into density at end

plt.close()

fig, axs = plt.subplots(3,3)
i = 0
j = 0
joint_prob = np.zeros((3,3))

for delta in [-1, 0, 1]:
    j = 0
    for k in capital_list:
        selection = plus_dict[(plus_dict[:, 4] == delta)*(plus_dict[:, 2] == k)]
        joint_prob[j, i] = np.sum(selection[:, -1])/np.sum(plus_dict[:,-1])
        print(f'(k,d) = {(k, delta)} with frequency {len(selection)} and probability {joint_prob[j, i]}')
        fit_x = np.repeat(selection[:,3], (selection[:, -1]*1000).astype(int))
        if len(fit_x) > 0:
            kernel = stats.gaussian_kde(fit_x, bw_method = 0.25/fit_x.std(ddof=1))
            dens = kernel(plot_x)
            #kern = KernelDensity(kernel='gaussian', bandwidth=0.7).fit(fit_x[:, np.newaxis])
            #dens = np.exp(kern.score_samples(plot_x[:, np.newaxis]))
            axs[j,i].plot(plot_x, dens)
            axs[j,i].set_title(f'k = {int(k)}, delta = {delta}')
        else:
            print('too small')
        j+= 1
    i += 1
axs[-1,0].set_xlabel(r'$\lambda$')
axs[-1, 0].set_ylabel('Density')
fig.suptitle(r'Estimated density of $\lambda$ given ($k$, $\delta$)')
plt.show()

In [ ]:
# Calculate probability distribution over capital endowments
np.sum(joint_prob, axis = 1)

### Stackelberg Scenario

#### Find Zeta

In [ ]:
weird_bot_1 = '0xa69babef1ca67a37ffaf7a485dfff3382056e78c' # Only does one half of transaction, always through Uniswap router
weird_bot_2 = '0xe841e62778d997729cbbe165b029ebb4af8a58ab'
attacks = pd.DataFrame({'User':[], TOK_A:[], TOK_B:[], 'Profit':[], 'Perc. Profit':[], 'Start':[], 'End':[], 'Idx1':[], 'Idx2':[], 'Txn Scale': []})
curr_df = master
i = -1
while i < len(curr_df)-3:
    i += 1
    user1 = [curr_df[f'User{TOK_A}'].iloc[i], curr_df[f'User{TOK_B}'].iloc[i]]
    user2 = [curr_df[f'User{TOK_A}'].iloc[i+2], curr_df[f'User{TOK_B}'].iloc[i+2]]
    
    users = user1 + user2
    user_overlap = list(set(user1) & set(user2))
    matchFlag = False
    
    '''
    # the user matching process is a little complicated because of certain bot "paired" behaviors observed in the data
    if len(user_overlap) >=1:
        user = user_overlap[0]
        matchFlag = True
    elif weird_bot_1 in users and weird_bot_2 in users: # weird bot combo matches
        user = weird_bot_1
        matchFlag = True
    
    '''
    # ok here's the dumb way:
    if user1[0] == user2[0] and user1[0] not in ROUTER: # first user matches
        user = user1[0]
        matchFlag = True
    elif user1[1] == user2[1] and user1[1] not in ROUTER: # second user matches
        user = user1[1]
        matchFlag = True
    elif weird_bot_1 in users and weird_bot_2 in users: # weird bot combo matches
        user = weird_bot_1
        matchFlag = True

    idx1 = curr_df.index[i]
    idx2 = curr_df.index[i+2]
    
    start = curr_df['timeStamp'].iloc[i]
    end = curr_df['timeStamp'].iloc[i+2]

    # if indices are close enough and users match, check transaction sizes
    if (idx2 - idx1 == 2) and (matchFlag == True) and ((end - start)//np.timedelta64(1, 's') == 0):
        pi_a = curr_df[TOK_A].iloc[i+2] + curr_df[TOK_A].iloc[i]
        pi_b = curr_df[TOK_B].iloc[i+2] + curr_df[TOK_B].iloc[i]
        scale = np.abs(curr_df[TOK_B].iloc[i]) # just the general size of the transaction

        err_a = np.abs(pi_a)/max(np.abs(curr_df[TOK_A].iloc[i]), 1e-4)
        err_b = np.abs(pi_b)/max(np.abs(curr_df[TOK_B].iloc[i]), 1e-4)
        
        flag1 = (curr_df['Transaction Type'].iloc[i] == 0) and (curr_df['Transaction Type'].iloc[i+2] == 0)
        flag2 = (curr_df['Transaction Type'].iloc[i] == 1) and (curr_df['Transaction Type'].iloc[i+2] == -1)

        if (flag1 or flag2) and (min(err_a, err_b) < 0.05):
            botflag = (curr_df['botFlag'].iloc[i] + curr_df['botFlag'].iloc[i+2])
            
            if curr_df['Transaction Type'].iloc[i] != 0:
                attackType = 1 # liquidity-based attack
            else:
                attackType = 0 # swap-based attack

            gas = curr_df['gasFee'].iloc[i] + curr_df['gasFee'].iloc[i+2]

            curr_profit = -(pi_a*curr_df['Market Price'].iloc[i] + pi_b) - gas # recall that a negative transaction is going away from Uniswap, aka TO the user
            rel = curr_profit/scale

            attacks = attacks.append({'User':user, TOK_A:-pi_a, TOK_B:-pi_b, 'Profit':curr_profit, 'Perc. Profit': rel, 'Idx1':idx1, 'Idx2':idx2, 'Start':start, 'End':end, 'Txn Scale': scale, 'botFlag': botflag, 'Type': attackType}, ignore_index = True)

print(len(attacks))

In [ ]:
swapBool = master['Transaction Type'] == 0
largeBool = np.abs(master[TOK_B]) > 5e5
arbitrageBool = master[TOK_B] * (np.array(master['Exchange Rate'])- np.array(master['Market Price'])) < 0
attackable = len(master[largeBool*swapBool*arbitrageBool])

In [ ]:
zeta = len(attacks)/attackable
print(f'Bots attack {round(zeta, 2)*100}% of attackable transactions.')

#### Re-run Calibration

Run the MFG calibration etc when the LPs anticipate Bots

In [ ]:
fees2, positions2 = calibrateAgents(nu, zeta, myT, myM)
#fees2, positions2 = calibrateAgents(nu, zeta, myT, myM, loadFile = False) # if you want to generate new MC trials

In [ ]:
# make a positions matrix
scale_factor = 1e4

pos_mat = np.zeros((1, nu.size))
pos_mat[0, :] = capital_list[0]/nu.size
for i in range(1, positions2.shape[0]):
    temp = np.zeros((1, nu.size))
    start = int(positions2[i, 0])
    end = int(positions2[i, 1])
    k = positions2[i, 2]
    temp[0, start:end] = k/(end - start)
    pos_mat = np.concatenate((pos_mat, temp), axis = 0)
# new below
temp = np.ones((positions2.shape[0], 1))*scale_factor
pos_mat = np.concatenate((pos_mat, temp), axis = 1)
ell_target = np.concatenate((ell0, [800*scale_factor]))
print(ell_target.shape)
pos_mat = np.transpose(pos_mat) # pos_mat.shape = (nu.size, number of types)
print(pos_mat.shape)

reg_sb2 = Ridge(positive=True, fit_intercept = False) # options are: Lasso, Ridge, LinearRegression
weights_sb2 = reg_sb2.fit(pos_mat, ell_target).coef_
print(np.sum(weights_sb2 > 0))
positions2 = np.concatenate((positions2, np.expand_dims(weights_sb2, axis = 1)), axis = 1)

reg_sb = LinearRegression(positive=True, fit_intercept = False) # options are: Lasso, Ridge, LinearRegression
weights_sb = reg_sb.fit(pos_mat, ell_target).coef_
print(f'R-score is {reg_sb.score(pos_mat, ell_target)}')

ell_new2 = np.matmul(pos_mat, weights_sb)
num_agents = ell_new2[-1]
ell_solved2 = ell_new2[:-1]
print(num_agents/scale_factor)

In [ ]:
print(f'Wasserstein-1 is {wasserstein_distance(nu.ticks[1:], nu.ticks[1:], ell0, ell_solved2)}')

In [ ]:
nu_temp = nu.copy()
nu_temp.resetLiquidity(ell_solved2, nu.pool_er)
liq_mfg2 = np.zeros(nu.size)
for i in range(nu.size):
    liq_mfg2[i] = nu_temp.K(i)
del nu_temp

In [ ]:
nu.plotEll(liq_mfg2, yLim = 5e7)

Estimate $\Theta$ for Stackelberg game

In [ ]:
plus_dict = positions2[positions2[:,-1] > 0]

In [ ]:
plot_x = np.arange(0, 6, 0.1) # list of points that we will plug into density at end

plt.close()

fig, axs = plt.subplots(3,3)
i = 0
j = 0
joint_prob = np.zeros((3,3))

for delta in [-1, 0, 1]:
    j = 0
    for k in capital_list:
        selection = plus_dict[(plus_dict[:, 4] == delta)*(plus_dict[:, 2] == k)]
        joint_prob[j, i] = np.sum(selection[:, -1])/np.sum(plus_dict[:,-1])
        print(f'(k,d) = {(k, delta)} with frequency {len(selection)} and probability {joint_prob[j, i]}')
        fit_x = np.repeat(selection[:,3], (selection[:, -1]*1000).astype(int))
        if len(fit_x) > 0 and len(selection) > 1:
            kernel = stats.gaussian_kde(fit_x, bw_method = 0.25/fit_x.std(ddof=1))
            dens = kernel(plot_x)
            #kern = KernelDensity(kernel='gaussian', bandwidth=0.7).fit(fit_x[:, np.newaxis])
            #dens = np.exp(kern.score_samples(plot_x[:, np.newaxis]))
            axs[j,i].plot(plot_x, dens)
            axs[j,i].set_title(f'k = {int(k)}, delta = {delta}')
        else:
            print('too small')
        j+= 1
    i += 1
axs[-1,0].set_xlabel(r'$\lambda$')
axs[-1, 0].set_ylabel('Density')
fig.suptitle(r'Estimated density of $\lambda$ given ($k$, $\delta$)')
plt.show()

In [ ]:
np.sum(joint_prob, axis = 1)

### 24 Hour Simulation

First let us check that we are working with the right initial distribution

In [ ]:
nu.plotLiquidity()

Now let's load the distribution for the next day. Format to ensure that both have consistent ticks and sizing.

In [ ]:
# load pre-saved liquidity distribution for November 30, 2023
date2 = '11302023'
curr_price = 2042.6874
intervals = np.load(f'data/{date2}ticks.npy')
tokA = np.load(f'data/{date2}tokA.npy')
tokB = np.load(f'data/{date2}tokB.npy')
nu_next = Pool(pool_type = len(intervals), mkt = curr_price, pool = curr_price, fee = 0.0005)
nu_next.setTicks(intervals)
nu_next.manualLiqSetAll(tokA, tokB)
nu_next.setABTicks()

In [ ]:
ell_next = np.zeros(nu_next.size)
for i in range(nu_next.size):
    ell_next[i] = nu_next.K(i)
nu_next.plotEll(ell_next, yLim = 5e7)

In [ ]:
start_tick = round(math.log((1/nu.ticks[-1])*(1e12), 1.0001)/10)*10
start_tick2 = round(math.log((1/nu_next.ticks[-1])*(1e12), 1.0001)/10)*10

hang = int((start_tick - start_tick2)/10)

nu_next.ticks = nu_next.ticks[:-hang]
nu_next.tokA = nu_next.tokA[:-hang]
nu_next.tokB = nu_next.tokB[:-hang]
nu_next.a_ticks = nu_next.a_ticks[:-hang]
nu_next.b_ticks = nu_next.b_ticks[:-hang]
nu_next.size = nu_next.size - hang

In [ ]:
future_ell = nu_next.tokA * nu_next.mkt_er + nu_next.tokB

In [ ]:
# change based on how you want to initialize the liquidity at time 0 (November 29)
#curr_ell = ell_solved # simple MFG
#z = 0 # simple MFG

curr_ell = ell_solved2 # Stackelberg
z = zeta # Stackelberg

In [ ]:
nu_trunc = nu.copy()
nu_trunc.resetLiquidity(curr_ell, nu.pool_er)

nu_trunc.ticks = nu_trunc.ticks[hang:]
nu_trunc.tokA = nu_trunc.tokA[hang:]
nu_trunc.tokB = nu_trunc.tokB[hang:]
nu_trunc.a_ticks = nu_trunc.a_ticks[hang:]
nu_trunc.b_ticks = nu_trunc.b_ticks[hang:]
nu_trunc.size = nu_trunc.size - hang
nu_trunc.idx = nu_trunc.idx - hang

fees_cut = fees[:, :, hang:]

np.mean(nu_trunc.ticks - nu_next.ticks)

In [ ]:
start_idx = master.index[(master['Exchange Rate'] < nu.pool_er+1)*(master['Exchange Rate'] > nu.pool_er-1)][0]

In [ ]:
end_idx = master.index[(master['Exchange Rate'] < nu_next.pool_er+1)*(master['Exchange Rate'] > nu_next.pool_er-1)][-1]

In [ ]:
comp_df = master[start_idx:end_idx]
comp_df = comp_df[comp_df['Transaction Type'] == 0]
comp_df

Okay, let's simulate:

In [ ]:
nu2 = nu_trunc.copy()

swapper = Swapper(swapper_type = 4, swap_max = 11.764, swap_min = 7.446)
#swapper = Swapper(swapper_type = 6, swap_max = mvkernel0)

swaps = []
pool = []
market = []
times = []
accuracy = 0
curr_time = comp_df['timeStamp'].iloc[0]

bot_capital = 5e6

bot_pi = 0
lp_pi = 0

rng = np.random.default_rng()
count = 0
print(f'zeta = {z}')

while count < len(comp_df) and curr_time <= comp_df['timeStamp'].iloc[-1]:
    if count % 100 == 0: print(count)
    
    curr_mkt = comp_df['Market Price'].iloc[count]
    if curr_mkt < 0:
        print('weird')
    nu2.setMktER(curr_mkt)
    xi = swapper.generateSwaps(nu2.pool_er - nu2.mkt_er, 1)[0]
    
    swaps.append(xi)
    times.append(curr_time)
    market.append(nu2.mkt_er)
    pool.append(nu2.pool_er)
    
    scale = bot_capital/(nu2.tokA[nu2.idx]*nu2.pool_er + nu2.tokB[nu2.idx])
    portion = bot_capital/(nu2.tokA[nu2.idx]*nu2.pool_er + nu2.tokB[nu2.idx] + bot_capital)
    
    threshold = botThreshold(nu2, nu2.K(nu2.idx) * scale)
    flag = np.random.binomial(1, z)
    
    if flag == 0 or (xi > threshold[0] and xi < threshold[1]): # no bot attack = swap procedes as usual
        nu2.swap(xi)
        lp_pi += nu.fee * np.abs(xi)
    else:
        print(f'bot attack: {round(portion, 4)*100}%')
        bot_pi += nu.fee*np.abs(xi)*portion
        lp_pi += nu.fee*np.abs(xi)*(1-portion)
        
    count += 1
    if count < len(comp_df): curr_time = comp_df['timeStamp'].iloc[count]
    
print(accuracy/count)

In [ ]:
print(f'Bot profit is {bot_pi} and LP profit is {lp_pi}')

In [ ]:
simEll = nu2.tokA * nu2.mkt_er + nu2.tokB

In [ ]:
print(f'MAPE for price evolution is {mape(pool, market)}') 
print(f'MSE for price evolution is {mse(pool, market)}') 

In [ ]:
print(f'Original MFG Wasserstein-1 is {wasserstein_distance(nu.ticks[1:], nu.ticks[1:], ell0, ell_solved)}')
print(f'Original Stackelberg Wasserstein-1 is {wasserstein_distance(nu.ticks[1:], nu.ticks[1:], ell0, ell_solved2)}')

In [ ]:
print(f'24-hour ahead Wasserstein-1 error is {wasserstein_distance(nu2.ticks[1:], nu2.ticks[1:], future_ell, simEll)}')

In [ ]:
print(f'24-hour ahead Wasserstein-1 error is {wasserstein_distance(nu2.ticks[1:], nu2.ticks[1:], future_ell, simEll)}')

In [ ]:
plt.close()
plt.plot(comp_df['timeStamp'], comp_df['Exchange Rate'], label = 'Real Pool Rate', color = 'blue', linewidth=1)
plt.plot(comp_df['timeStamp'], comp_df['Market Price'], label = 'Market Rate', color = 'black', linewidth=1)
plt.plot(times, pool, label = 'Simulated Pool Rate', color = 'red', linewidth=1.5)
plt.xlabel('Time')
plt.ylabel('USDC/ETH')
plt.title('Simulated Exchange Rate Dynamics')
plt.legend()
plt.show()

In [ ]:
nu2.resetLiquidity(simEll, nu2.pool_er)
simliq = np.zeros(nu2.size)
for i in range(nu2.size):
    simliq[i] = nu2.K(i)

In [ ]:
nu2.plotEll(simliq, yLim = 5e7)